In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd

import sys
sys.path.append("../")
from bargains_linear import * 

In [2]:
def gen_data(params,nobs=100,var=5):
    wtp,cost,beta1,beta2,emc1,emc2=params
    emc = emc1,emc2
    passive_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2']) 
    active_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'])
    seq_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'])

    for i in range(0,nobs):
        #sequential
        mc1 = np.random.normal(loc=0, scale=var, size=(2,)) + np.array([emc1,emc2])
        mc2 = np.random.normal(loc=0, scale=1, size=(2,))
        mc = mc1+mc2
        
        #passive results
        phi1,phi2 =simult_bargain(25,25, cost,wtp,mc, active=False,betas=[beta1,beta2])
        phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp,mc)
        row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1]]])
        passive_result = pd.concat( [passive_result, row] )
        
        #seq results
        phi1,phi2 = seq_bargain(25,cost,wtp,mc,betas=[beta1,beta2])
        phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp,mc)
        row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1]]])
        seq_result = pd.concat( [seq_result, row] )


        #active results
        phi1,phi2 =simult_bargain(25,25, cost,wtp,mc, active=True,betas=[beta1,beta2])
        phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp,mc)
        row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1]]])
        active_result = pd.concat( [active_result, row] )



    return seq_result, active_result,passive_result


param_list = [[25,5,.5,.5,0,0],[20,5,.5,.5,0,0],[25,10,.5,.5,0,0],[25,5,.5,.1,0,0],[25,5,.1,.5,0,0]]

for i in range(len(param_list)):
    seq_result,active_result,passive_result = gen_data(param_list[i],nobs=10000,var=5)
    seq_result.to_csv('fake_data/seq_data_%s.csv'%i)
    active_result.to_csv('fake_data/active_data_%s.csv'%i)
    passive_result.to_csv('fake_data/passive_data_%s.csv'%i)

In [3]:
#calculate moment conditions for the linear model...
def solve_parameters_obj(data,params, active=False,seq=False):
    wtp,cost,beta1,beta2,mc1,mc2 = params
    
    data_moments = list(data.mean())
    phi1,phi2 = None,None
    if seq:
        phi1,phi2 = seq_bargain(25, cost, wtp, [mc1,mc2],betas=[beta1,beta2])
    elif active:
        phi1,phi2 = simult_bargain(25,25, cost, wtp, [mc1,mc2],active=True,betas=[beta1,beta2])
    else:
        phi1,phi2 = simult_bargain(25,25, cost, wtp, [mc1,mc2],active=False,betas=[beta1,beta2])
    
    phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp,[mc1,mc2])

    computed_moments = [phi1,phi2,p1,p2,s1,s2]
    
    obj = 0
    for i in range(6):
        obj = obj + (data_moments[i] - computed_moments[i])**2
    
    return obj

print(seq_result)

print( solve_parameters_obj(seq_result,[25,5,.5,.5,0,0], seq=True) )
print( solve_parameters_obj(seq_result,[25,5,.5,.5,0,0], active=True) )

         phi1          phi2         p1         p2        s1        s2  \
0   31.802015  1.772408e-04  24.144108  13.824613  0.000000  1.000000   
0   31.405547  9.592327e-14  24.655929  14.124919  0.000000  1.000000   
0   11.282738  2.004805e+01  15.345241  11.785632  0.144039  0.855961   
0   41.781075  2.842171e-14  33.016460  21.993170  0.000000  1.000000   
0   29.659867  1.853698e-04  28.163921  16.782707  0.000000  1.000000   
..        ...           ...        ...        ...       ...       ...   
0   20.246135  2.782302e+01  29.447464  26.751455  0.230399  0.769601   
0   41.028572  2.842171e-14  31.183716  19.966267  0.000000  1.000000   
0   44.982893  2.842171e-14  31.356715  21.211183  0.000000  1.000000   
0   33.162483  2.131628e-14  29.434569  19.896970  0.000000  1.000000   
0   31.569474  4.973799e-14  29.650229  19.205758  0.000000  1.000000   

          c1         c2  
0  -3.700282  -1.912291  
0  -0.880291  -0.361895  
0   1.828107 -15.461030  
0  -3.336810   6.73

In [4]:
def solve_parameters(data,active=False,seq=False): 
    obj = lambda params: solve_parameters_obj(data,params,active=active,seq=seq)
    params = minimize(obj, [25,5,.5,.5,0,0], options={'fatol':1e-2,'xatol':1e-2,'maxiter':20},method='Nelder-Mead').x # 
    return params

print(solve_parameters(seq_result,active=False,seq=True))
print(solve_parameters(active_result,active=True,seq=False))
print(solve_parameters(passive_result,active=False,seq=False))

[2.65511526e+01 4.34035724e+00 3.74484105e-01 6.50804319e-01
 7.87871259e-05 3.35439977e-04]
[ 3.18175583e+01  5.19684499e+00  4.03292181e-01  4.73593964e-01
 -6.99759945e-04  5.42524005e-04]
[2.63657407e+01 6.27777778e+00 4.18750000e-01 3.53935185e-01
 3.05555556e-04 3.61111111e-04]
